# Submission for Research Assistant position at ISI
### Table understanding group

***
**Submission by:**
Abhinav Singh<br>
**USC ID:** 3040418309 | **USC Email ID:** asingh45@usc.edu

In [1]:
import pandas as pd

### Read canonical table with 0th row as header and csv delimiter ','

In [2]:
canonical = pd.read_csv('Canonical.csv', header=[0], delimiter=',')
canonical

,Country,Indicator,Crop,Year,Value
0,Ethiopia,production,maize,2015,1
1,Ethiopia,production,wheat,2015,2
2,Ethiopia,production,maize,2016,3
3,Ethiopia,production,wheat,2016,4
4,Ethiopia,area,maize,2015,5
5,Ethiopia,area,wheat,2015,6
6,Ethiopia,area,maize,2016,7
7,Ethiopia,area,wheat,2016,8
8,Somalia,production,maize,2015,9
9,Somalia,production,wheat,2015,10


### Read table1.csv with 0th row as header and csv delimiter ','

In [3]:
table1_csv = pd.read_csv('Table1.csv', header=[0], delimiter=',')
table1_csv

,Country,production,area,Crop,Year
0,Ethiopia,1,5,maize,2015
1,Ethiopia,2,6,wheat,2015
2,Ethiopia,3,7,maize,2016
3,Ethiopia,4,8,wheat,2016
4,Somalia,9,13,maize,2015
5,Somalia,10,14,wheat,2015
6,Somalia,11,15,maize,2016
7,Somalia,12,16,wheat,2016


### Read table2.csv with 0th row as header and csv delimiter ','

In [4]:
table2_csv = pd.read_csv('Table2.csv', header=[0], delimiter=',')
table2_csv

,Country,Indicator,Crop,2015,2016
0,Ethiopia,production,maize,1,3
1,Ethiopia,production,wheat,2,4
2,Ethiopia,area,maize,5,7
3,Ethiopia,area,wheat,6,8
4,Somalia,production,maize,9,11
5,Somalia,production,wheat,10,12
6,Somalia,area,maize,13,15
7,Somalia,area,wheat,14,16


### Read table3.csv with 0th and 1st row as header to form a multiindex column header and csv delimiter ','

In [5]:
table3_csv = pd.read_csv('Table3.csv', header=[0, 1], delimiter=',')
table3_csv

- production      area           -
    Country       2015 2016 2015 2016   Crop
0  Ethiopia          1    3    5    7  maize
1  Ethiopia          2    4    6    8  wheat
2   Somalia          9   11   13   15  maize
3   Somalia         10   12   14   16  wheat

### Lift Operator documentation and working

The lift operator below is generic enough to handle any green block size as long as it is continuous. The function takes input the dataframe which needs to be transformed, the lift_column as a string which is one of the columns of the dataframe, green_block which is a tuple of two integers specifying the column indices of the green cells inclusive of left value and exclusive of right value and finally the remove_green_header parameter which decides whether the transformed table will have an extra header row or will it be squeezed into a single row only. This is a boolean variable. This parameter needs to be used carefully based on the requirement and usecase. 

This function returns the transformed table and the new green block in the transformed table.

To generate the transformed table, I first process the column header information to generate new column name headers and create an empty dataframe in which I insert data. 

To create new column names, it is very easy in the case when remove_green_header is set to True. For that the new columns are all the columns leaving the green column and the lift column along with all the distinct values which are in that particular lift column. This is easily handled in the code. 

For the case where we need to preserve the green header information, we have to add an extra layer of column header in dataframe for every level of transformation. This is implemented using MultiIndex in dataframes. MultiIndex can be created using a list of tuples. 

At every layer of transformation, the new column header consists of one more layer than the previous column header where each distinct value of the lift_column is mapped/associated with each green header column in the block. For rest of the columns, an extra layer is inserted but the value of that level is just instantiated with a hyphen (-) for simplicity. 

Once this empty data frame is created with these modified multi index column headers, we have to insert the data in this modified schema. To identify the groups based on the remaining columns(total cols - (lift_column + green column headers)), I perform a group by operation on the original dataset and generate these groups. For each of the group generated, the key data is inserted in the remaining columns on which the group by was performed, for all the newly added columns, the data is inserted using the grouped data into respective multi index column headers. 

After inserting all the data, the new data frame and the updated green block is returned by the function. I make sure that the newly inserted columns are always towards the extreme right of the table to find out the new green block easily. 

In [6]:
def lift_operator(df, lift_column, green_block, remove_green_header):
    lift_column = tuple(['-' for i in range(len(df.columns[0]) - 1)]) + (lift_column,) if type(df.columns[0]) is tuple else lift_column
    
    cols = [e for e in df.columns if e != lift_column and e not in df.columns[green_block[0]:green_block[1]]]
    
    if not cols:
        print('No columns left in the remaining table to group by from.')
        return
    
    added_cols = sorted(df[lift_column].unique())
    green_cols = list(df.columns[green_block[0]:green_block[1]])

    added_cols = [str(e) for e in added_cols]
    green_cols = [str(e) if type(e) is not tuple else e for e in green_cols]
    
    if not remove_green_header:
        new_added_cols_list = list()
        for col in cols:
            new_added_cols_list.append(('-',) + (col if type(col) is tuple else (col,)))
        for col in added_cols:
            for green_col in green_cols:
                new_added_cols_list.append((col,) + (green_col if type(green_col) is tuple else (green_col,)))
        new_cols = pd.MultiIndex.from_tuples(new_added_cols_list)
    else:
        new_cols = cols + added_cols
        
    new_df = pd.DataFrame(columns=new_cols)
    
    grouped_df = df.groupby(cols)
    for key, item in grouped_df:
        data_dict = dict()
        for i, col in enumerate(cols):
            if not remove_green_header:
                data_dict[('-',) + (col if type(col) is tuple else (col,))] = key[i] if len(cols) > 1 else key
            else:
                data_dict[col] = key[i] if len(cols) > 1 else key

        for i, row in item.iterrows():
            for green_col in green_cols:
                if not remove_green_header:
                    data_dict[(str(row[lift_column]),) + (green_col if type(green_col) is tuple else (green_col,))] = row[green_col]
                else:
                    data_dict[str(row[lift_column])] = row[green_col]

        new_df = new_df.append(data_dict, ignore_index=True)

    new_green_block = (len(cols), len(new_cols))
    
    return new_df, new_green_block

### Transformation 1 (Canonical -> Table1)

For the first transformation from Canonical table to Table1, we set the green_block as (4,5). The green_block is a tuple which tells the continuous block size of green cells, basically the coordinates of the columns of the green cells. The range is inclusive of the left value but exclusive of the right value of the tuple. These indexes of columns are 0-indexed, meaning the left most column will have index 0.

The lift_column is set as 'Indicator' for this transformation and remove_green_header parameter is set to True as we want to remove the header of the green block i.e. Value.

In [7]:
start_col, end_col = 4, 5
green_block = (start_col, end_col)
remove_green_header = True

table1, table1_green_block = lift_operator(canonical, lift_column='Indicator', green_block=green_block, remove_green_header=True)


In [8]:
print('Table 1 after transformation is as follows: ')
table1

Table 1 after transformation is as follows: 


,Country,Crop,Year,area,production
0,Ethiopia,maize,2015,5,1
1,Ethiopia,maize,2016,7,3
2,Ethiopia,wheat,2015,6,2
3,Ethiopia,wheat,2016,8,4
4,Somalia,maize,2015,13,9
5,Somalia,maize,2016,15,11
6,Somalia,wheat,2015,14,10
7,Somalia,wheat,2016,16,12


In [9]:
print('New value of green_block for table 1 is:', table1_green_block)

New value of green_block for table 1 is: (3, 5)


### Transformation 2 (Canonical -> Table2)

For the second transformation from Canonical table to Table1, we set the green_block as (4,5) again same as before. The lift_column is set as 'Year' for this transformation and remove_green_header parameter is set to True as we want to remove the header of the green block i.e. Value.

In [10]:
start_col, end_col = 4, 5
green_block = (start_col, end_col)
remove_green_header = True

table2, table2_green_block = lift_operator(canonical, lift_column='Year', green_block=green_block, remove_green_header=True)


In [11]:
print('Table 2 after transformation is as follows: ')
table2

Table 2 after transformation is as follows: 


,Country,Indicator,Crop,2015,2016
0,Ethiopia,area,maize,5,7
1,Ethiopia,area,wheat,6,8
2,Ethiopia,production,maize,1,3
3,Ethiopia,production,wheat,2,4
4,Somalia,area,maize,13,15
5,Somalia,area,wheat,14,16
6,Somalia,production,maize,9,11
7,Somalia,production,wheat,10,12


In [12]:
print('New value of green_block for table 2 is:', table2_green_block)

New value of green_block for table 2 is: (3, 5)


### Transformation 3 (Table2 -> Table3)

For the third transformation from Table2 to Table3, we use the green_block which was returned on getting table2 transformation. The lift_column is set as 'Indicator' for this transformation and remove_green_header parameter is set to False as we want a multi layered header for this transformation which retains the column header before the transformation.

In [13]:
table3, table3_green_block = lift_operator(table2, lift_column='Indicator', green_block=table2_green_block, remove_green_header=False)


In [14]:
print('Table 3 after transformation is as follows: ')
table3

Table 3 after transformation is as follows: 


-        area      production     
    Country   Crop 2015 2016       2015 2016
0  Ethiopia  maize    5    7          1    3
1  Ethiopia  wheat    6    8          2    4
2   Somalia  maize   13   15          9   11
3   Somalia  wheat   14   16         10   12

In [15]:
print('New value of green_block for table 3 is:', table3_green_block)

New value of green_block for table 3 is: (2, 6)


### Extra transformation over table3.

I went ahead and tried to further transform this table3 by lifting the 'Crop' column again. The code is written in such a way that it is able to keep lifting a column till the time just one pure column is not left in entire table. For this transformation, I use table3 and the green block returned by table 3 transformation and lift_column as 'Crop'. I won't remove the green header for this transformation to see another 3rd level of header added to the table.

In [16]:
table4, table4_green_block = lift_operator(table3, 'Crop', table3_green_block, remove_green_header=False)


In [17]:
print('Table 4 after transformation is as follows: ')
table4

Table 4 after transformation is as follows: 


- maize                      wheat                     
          -  area      production       area      production     
    Country  2015 2016       2015 2016  2015 2016       2015 2016
0  Ethiopia     5    7          1    3     6    8          2    4
1   Somalia    13   15          9   11    14   16         10   12

In [18]:
print('New value of green_block for table 4 is:', table4_green_block)

New value of green_block for table 4 is: (1, 9)


### Extra transformation over table3 with an updated 'Crop' value in column. 

I updated one of the row's 'Crop' value to 'rice' in table3 to see the updated transformation. The same function was able to handle this case as well by inserting Nan values to the transformed table for which data does not exist. The below updation of table3 and the subsequent transformation demonstrates the same.

In [19]:
table3[('-', 'Crop')][3] = 'rice'
print('Updated table 3 is as follows:')
table3

Updated table 3 is as follows:


-        area      production     
    Country   Crop 2015 2016       2015 2016
0  Ethiopia  maize    5    7          1    3
1  Ethiopia  wheat    6    8          2    4
2   Somalia  maize   13   15          9   11
3   Somalia   rice   14   16         10   12

In [20]:
table4_updated, table4_updated_green_block = lift_operator(table3, 'Crop', table3_green_block, remove_green_header=False)

In [21]:
print('Table 4 updated after transformation is as follows: ')
table4_updated

Table 4 updated after transformation is as follows: 


- maize                      rice                      wheat       \
          -  area      production      area      production       area        
    Country  2015 2016       2015 2016 2015 2016       2015 2016  2015 2016   
0  Ethiopia     5    7          1    3  NaN  NaN        NaN  NaN     6    8   
1   Somalia    13   15          9   11   14   16         10   12   NaN  NaN   

                   
  production       
        2015 2016  
0          2    4  
1        NaN  NaN

In [22]:
print('New value of green_block for table 4 updated is:', table4_updated_green_block)

New value of green_block for table 4 updated is: (1, 13)
